In [1]:
import pandas as pd

df = pd.read_csv("transfer_10.csv").dropna()
df.shape

(44563, 24)

Let's remove instances that were in samples and instances where sim < 0.8:

In [2]:
c1 = ~df.was_in_samples
c2 = df.sim >= 0.8

df = df.loc[c1 & c2, :]


Let's prepare a subset of 25k instances and calculate the duration:

In [3]:
subset = df.iloc[:25500].reset_index(drop=True)
((subset.end - subset.start).cumsum()/3600).values[-1]

114.72261111111041

Ok, we have 114.7 hours of data. Let's randomize the row order instead of doing train-dev-test split:

In [4]:
subset = subset.sample(frac=1).reset_index(drop=True)

In [6]:
subset["split"] = ["train"] * 24500 + ["test"] * 500 + ["dev"] * 500

In [10]:
condition = subset.split == "test"
subsubset = subset.loc[condition, :]
(subsubset.end-subsubset.start).sum()/3600

2.2508305555555506

In [24]:
train = subset.loc[
    subset.split=="train",  ['hashname',
                            'human_transcript',
                            'unnormalized_transcript']
                    ]
test = subset.loc[
    subset.split=="test",  ['hashname',
                            'human_transcript',
                            'unnormalized_transcript']
                    ]
dev = subset.loc[
    subset.split=="dev",  ['hashname',
                            'human_transcript',
                            'unnormalized_transcript']
                    ]

train.to_csv("17_train.csv", index=False)
dev.to_csv("17_dev.csv", index=False)
test.to_csv("17_test.csv", index=False)

# Implementing the unnormalized preprocessing:

In [29]:
import parse
s = df.unnormalized_transcript[1]
s

'u svezi sa člankom 190. Poslovnika Hrvatskog sabora. Materijal vam je dostavljen u pretince. Ove zakonske prijedloge raspravljat ćemo sukladno odredbama Poslovnika koje se odnose na drugo čitanje. Amandmani se mogu podnositi do kraja rasprave sukladno članku 197. Poslovnika. Raspravu su proveli Odbor'

In [31]:
p = parse.compile("{hit:d}.")
for seg in s.split():
    result = p.parse(seg)
    if result:
        print(result)

<Result () {'hit': 190}>
<Result () {'hit': 197}>


In [53]:
def process(text:str):
    from parse import compile
    from string import punctuation
    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans('', '', punctuation)).lower())
    return " ".join(out_list)



train['unnormalized_transcript'] = train.unnormalized_transcript.apply(process)
test['unnormalized_transcript'] = test.unnormalized_transcript.apply(process)
dev['unnormalized_transcript'] = dev.unnormalized_transcript.apply(process)
train.to_csv("17_train.csv", index=False)
dev.to_csv("17_dev.csv", index=False)
test.to_csv("17_test.csv", index=False)


In [52]:
df.human_transcript[0]

'konačni prijedlog zakona o izmjenama i dopunama zakona o doprinosima drugo čitanje pzbrdevetnaest i konačni prijedlog zakona o porezu na dohodak drugo čitanje pzbrdvadeset tri predlagatelj je vlada republike hrvatske na temelju članka osamdeset pet ustava erha i članka sto sedamdeset dva poslovnika'

In [57]:
pd.set_option('display.max_colwidth', None)
test.unnormalized_transcript.head(20)

24500                                                                                                                        ribarsku brodicu i sad što se događa imamo sustav enormnog licemjerja licemjerja u kojem bankar iz hbora koji je odobrio kredit od 115 milijuna kuna eph holdingu neće snositi nikakvu odgovornost a
24501                                                          bi ga zaista preciznije definirati bruto površinu na građevnoj čestici no ono što još jedino koristim priliku i ovu repliku da jednu dodatnu sugestiju a to je odnos prostornih planova većeg reda prema prostornim planovima nižeg reda vrlo često u praksi imate
24502                              uz radno vrijeme moramo biti svjesni da to ljude neće destimulirati na žalost i neće ih staviti u bolju poziciju u hrvatskoj i natjerati da poštuju zakone zbog toga što uzrok problema nije riješen znači mi u hrvatskoj imamo kronični nedostatak zaposlenih radnika u cestovnom prijevozu i
24503                             